# <span style='color:red'> NETFLIX </span> 
## EXPLORATORY DATA ANALYSIS

### Importing Spark and Netflix Dataset

In [1]:
#starting spark session
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Netflix').getOrCreate()

In [4]:
#calling netflix data set as df
df = spark.read.csv('netflix_titles.csv',header=True,inferSchema=True)

In [5]:
#inspecting schema
df.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



## Columns groups based profiling to better understand our data

we will examine columns which we can group by for further analysis while acknowledging columns which will need more complex tranformation in order to group by

In [111]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first

print ("Summary of columns show_id,type,title,director,cast,rating,listed_in,description")
df.select("show_id",'type','title','director','cast','rating','listed_in','description').summary().show()

print("Checking for nulls on columns show_id,type,title,director,cast,rating,listed_in,description:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in ["show_id",'type','title','director','cast','rating','listed_in','description']]).show()

print("Checking amount of distinct values in columns show_id,type,title,director,cast,rating,listed_in,description:")
df.select([countDistinct(c).alias(c) for c in ['type','title','director','cast','rating','listed_in','description']]).show()

print ("Most and least frequent occurrences for type,director,cast,rating columns:")
typeDF = df.groupBy("type").agg(count(lit(1)).alias("Total"))
directorDF    = df.groupBy("director").agg(count(lit(1)).alias("Total"))
castDF      = df.groupBy("cast").agg(count(lit(1)).alias("Total"))
ratingDF      = df.groupBy("rating").agg(count(lit(1)).alias("Total"))

leastFreqtype    = typeDF.orderBy(col("Total").asc()).first()
mostFreqtype     = typeDF.orderBy(col("Total").desc()).first()
leastFreqdirector       = directorDF.orderBy(col("Total").asc()).first()
mostFreqdirector        = directorDF.orderBy(col("Total").desc()).first()
leastFreqcast         = castDF.orderBy(col("Total").asc()).first()
mostFreqcast          = castDF.orderBy(col("Total").desc()).first()
leastFreqrating         = ratingDF.orderBy(col("Total").asc()).first()
mostFreqrating          = ratingDF.orderBy(col("Total").desc()).first()


display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqtype", "mostFreqtype", "leastFreqdirector", "mostFreqdirector", \
       "%s (%d occurrences)" % (leastFreqtype["type"], leastFreqtype["Total"]), \
       "%s (%d occurrences)" % (mostFreqtype["type"], mostFreqtype["Total"]), \
       "%s (%d occurrences)" % (leastFreqdirector["director"], leastFreqdirector["Total"]), \
       "%s (%d occurrences)" % (mostFreqdirector["director"], mostFreqdirector["Total"]))))
display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqcast", "mostFreqcast", "leastFreqrating", "mostFreqrating", \
       "%s (%d occurrences)" % (leastFreqcast["cast"], leastFreqcast["Total"]), \
       "%s (%d occurrences)" % (mostFreqcast["cast"], mostFreqcast["Total"]), \
       "%s (%d occurrences)" % (leastFreqrating["rating"], leastFreqrating["Total"]), \
       "%s (%d occurrences)" % (mostFreqrating["rating"], mostFreqrating["Total"]))))

Summary of columns show_id,type,title,director,cast,rating,listed_in,description
+-------+----------------+-------+---------------------------------+--------------------+--------------------+-----------------+---------------+--------------------+
|summary|         show_id|   type|                            title|            director|                cast|           rating|      listed_in|         description|
+-------+----------------+-------+---------------------------------+--------------------+--------------------+-----------------+---------------+--------------------+
|  count|            7788|   7788|                             7787|                5398|                7070|             7780|           7786|                7786|
|   mean|            null|   null|               1084.7272727272727|                null|                null|          2015.75|           null|  2014.6666666666667|
| stddev|            null|   null|               1096.7795668145072|                null|


| leastFreqtype | mostFreqtype | leastFreqdirector | mostFreqdirector |
|----|----|----|----|
| TV Show (2410 occurrences) | Movie (5378 occurrences) | Gaby Dellal (1 occurrences) | None (2390 occurrences) |



| leastFreqcast | mostFreqcast | leastFreqrating | mostFreqrating |
|----|----|----|----|
| Tedd Chan, Stella Chung, Henley Hii, Lawrence Koh, Tommy Kuan, Josh Lai, Mark Lee, Susan Leong, Benjamin Lim (1 occurrences) | None (718 occurrences) | November 1, 2020 (1 occurrences) | TV-MA (2854 occurrences) |


### 1. Distribution of TV Shows and Movies

Content on Netflix is classified mainly between TV Shows and Movies with **31%** being TV shows and **69%** being Movies.
<br>
<br>
Netflix relies more on externally productions on top of Netflix Originals to build their Movie library where as it focuses more on Netflix Originals for its TV show library thus explaining the split

In [6]:
#content type in Netlfix is primarly distributed into two types - tv shows and movies
by_type = df.groupBy('type').count()
by_type.show()

+-------------+-----+
|         type|count|
+-------------+-----+
|         null|    1|
|      TV Show| 2410|
|        Movie| 5377|
|William Wyler|    1|
+-------------+-----+



on examining William Wyler we observe it is actually a Movie and replace the type

In [7]:
from pyspark.sql.functions import *

In [8]:
df = df.withColumn('type',regexp_replace('type','William Wyler','Movie'))

dropping null value by defining a threshold

In [9]:
df = df.na.drop(thresh=2)

In [10]:
by_type = df.groupBy('type').count()
by_type.show()

+-------+-----+
|   type|count|
+-------+-----+
|TV Show| 2410|
|  Movie| 5378|
+-------+-----+



In [11]:
#total sum of counts = 7787
sum_of_counts = by_type.select(sum(by_type['count'])).collect()[0][0]

#share of total type
by_type.withColumn('share %',round(by_type['count']/sum_of_counts*100,2)).show()

+-------+-----+-------+
|   type|count|share %|
+-------+-----+-------+
|TV Show| 2410|  30.95|
|  Movie| 5378|  69.05|
+-------+-----+-------+



**1.1** **Total Content Added Yearly**
<br>
<br>
As Netflix grows more popular year after year, we see an increase in content added growing **75%** from 2017 to 2019. The dip in 2020 **(6%)** is most probably because of the halt in production due to the pandemic. 

In [12]:
#extracting year from column date_added
df_mod = df.withColumn('added_year',split(col('date_added'),',')[1])

In [13]:
#creating temporary table for sql
df_mod.createOrReplaceTempView('df_mod')

spark.sql('SELECT added_year,COUNT(*) as count FROM df_mod \
           WHERE added_year = 2017 OR added_year = 2018 OR added_year = 2019 OR added_year = 2020 or added_year = 2021 \
           GROUP BY added_year \
           ORDER BY added_year').show()

+----------+-----+
|added_year|count|
+----------+-----+
|      2017| 1223|
|      2018| 1683|
|      2019| 2151|
|      2020| 2003|
|      2021|  115|
+----------+-----+



% increase from 2017 to 2019

In [14]:
# % increase from 2017 to 2019
(2151/1223-1)*100

75.8789860997547

% drop from 2019 to 2020

In [15]:
# % drop from 2019 to 2020
(2003/2151-1)*100

-6.880520688052072

**1.2** **TV Show Content Added Yearly**
<br>
<br>
increase of **92%** from 2017 to 2020
<br>
increase of **6%** from 2019 to 2020

In [16]:
# filtering for tv_shows
df_mod_tv_show = df_mod.filter(df['type']=='TV Show')

df_mod_tv_show.createOrReplaceTempView('df_mod_tv_show')

spark.sql('SELECT type,added_year,COUNT(*) as count FROM df_mod_tv_show \
           WHERE added_year = 2017 OR added_year = 2018 OR added_year = 2019 OR added_year = 2020 or added_year = 2021 \
           GROUP BY type,added_year ORDER BY added_year').show()

+-------+----------+-----+
|   type|added_year|count|
+-------+----------+-----+
|TV Show|      2017|  361|
|TV Show|      2018|  430|
|TV Show|      2019|  655|
|TV Show|      2020|  696|
|TV Show|      2021|   29|
+-------+----------+-----+



% increase from 2017 to 2020

In [17]:
# % increase from 2017 to 2020
((696/361)-1)*100

92.797783933518

% increase from 2019 to 2020

In [18]:
# % increase from 2019 to 2020
((696/655)-1)*100

6.259541984732819

**1.3** **Movie Content Added Yearly**
<br>
<br>
increase of **52%** from 2017 to 2020
<br>
drop of **12%** from 2019 to 2020

In [19]:
# filtering for movies
df_mod_movie = df_mod.filter(df['type']=='Movie')

df_mod_movie.createOrReplaceTempView('df_mod_movie')

spark.sql('SELECT type,added_year,COUNT(*) as count FROM df_mod_movie \
WHERE added_year = 2017 OR added_year = 2018 OR added_year = 2019 OR added_year = 2020 or added_year = 2021 \
GROUP BY type,added_year ORDER BY added_year').show()


+-----+----------+-----+
| type|added_year|count|
+-----+----------+-----+
|Movie|      2017|  862|
|Movie|      2018| 1253|
|Movie|      2019| 1496|
|Movie|      2020| 1307|
|Movie|      2021|   86|
+-----+----------+-----+



% increase from 2017 to 2020

In [20]:
#increase from 2017 to 2020
((1307/862)-1)*100

51.624129930394425

% drop from 2019 to 2020

In [21]:
#decrease from 2019 to 2020
((1307/1496)-1)*100

-12.633689839572193

**1.4 Average Duration**

In [22]:
#average duration of movies and average seasons of tv shows

from pyspark.sql.types import IntegerType,ShortType,StringType,StructField
from pyspark.sql.functions import when

df_mod_movie_duration = df_mod_movie.withColumn('duration_int',(split(df['duration'],' ')[0]).cast('Integer'))
df_mod_movie_duration.withColumn('duration_int',when(col('duration_int')>350,0).otherwise(col('duration_int'))) \
                     .orderBy(df_mod_movie_duration['duration_int'].desc())\
                     .select(round(mean('duration_int'),2).alias('AVG MOVIE DURATION')).show()


df_mod_tv_show_season = df_mod_tv_show.withColumn('seasons',split('duration',' ')[0].cast('Integer'))
df_mod_tv_show_season.select(round(mean('seasons'),2).alias('AVG TV SHOW SEASONS')).show()

+------------------+
|AVG MOVIE DURATION|
+------------------+
|             99.31|
+------------------+

+-------------------+
|AVG TV SHOW SEASONS|
+-------------------+
|               1.78|
+-------------------+



### 2. Popular Actors and Directors

In [23]:
# popular directors by movies

df_mod_movie.groupBy('director').count().\
orderBy(col('count').desc()).filter(col('director')!='null').show(10,0)

+----------------------+-----+
|director              |count|
+----------------------+-----+
|Raúl Campos, Jan Suter|18   |
|Marcus Raboy          |15   |
|Jay Karas             |14   |
|Cathy Garcia-Molina   |13   |
|Youssef Chahine       |12   |
|Martin Scorsese       |12   |
|Jay Chapman           |12   |
|Steven Spielberg      |10   |
|David Dhawan          |9    |
|Kunle Afolayan        |8    |
+----------------------+-----+
only showing top 10 rows



**2.1 List of movies by Raul Campos**

In [24]:
# movies by Raúl Campos, Jan Suter
number_of_movies = df.filter(col('director')=='Raúl Campos, Jan Suter').select('title').count()

df.filter(col('director')=='Raúl Campos, Jan Suter').select('title').show(number_of_movies, 0)

+-------------------------------------------+
|title                                      |
+-------------------------------------------+
|Alan Saldaña: Mi vida de pobre             |
|Arango y Sanint: Ríase el show             |
|Carlos Ballarta: Furia Ñera                |
|Coco y Raulito: Carrusel de ternura        |
|Daniel Sosa: Sosafado                      |
|Fernando Sanjiao: Hombre                   |
|Jani Dueñas: Grandes fracasos de ayer y hoy|
|Luciano Mellera: Infantiloide              |
|Malena Pichot: Estupidez compleja          |
|Mea Culpa                                  |
|Natalia Valdebenito: El especial           |
|Ricardo O'Farrill Abrazo Genial            |
|Ricardo O'Farrill: Abrazo navideño         |
|Ricardo Quevedo: Hay gente así             |
|Sebastián Marcelo Wainraich                |
|Sofía Niño de Rivera: Exposed              |
|Sofía Niño de Rivera: Selección Natural    |
|Todo lo que sería Lucas Lauriente          |
+---------------------------------

**2.2 Looking at my personal favourite Martin Scoreses**

In [25]:
# movies by Martin Scorsese
number_of_movies = df.filter(col('director')=='Martin Scorsese').select('title').count()

df.filter(col('director')=='Martin Scorsese').select('title').show(number_of_movies,0)

+-----------------------------------------------------------+
|title                                                      |
+-----------------------------------------------------------+
|Alice Doesn't Live Here Anymore                            |
|Gangs of New York                                          |
|GoodFellas                                                 |
|Hugo                                                       |
|Mean Streets                                               |
|No Direction Home: Bob Dylan                               |
|Raging Bull                                                |
|Rolling Thunder Revue: A Bob Dylan Story by Martin Scorsese|
|Taxi Driver                                                |
|The Departed                                               |
|The Irishman                                               |
|Who's That Knocking at My Door?                            |
+-----------------------------------------------------------+



**Martin Scorsese often works with the same actors**

In [26]:
#extracting cast members who have worked with Martin Scorsese
martin_cast = []
for num in range(1,df.filter(col('director')=='Martin Scorsese').select('cast').count()):
    for i in df.filter(col('director')=='Martin Scorsese').withColumn('cast',split('cast',','))\
    .select('cast').collect()[num][0]:
        martin_cast.append(i)
        

In [27]:
# top 3 actors who have worked with Martin
spark.createDataFrame(martin_cast,'String')\
.toDF('actors').groupBy('actors').count().orderBy(col('count').desc()).show(3)

+--------------+-----+
|        actors|count|
+--------------+-----+
|Robert De Niro|    5|
| Harvey Keitel|    4|
|     Joe Pesci|    3|
+--------------+-----+
only showing top 3 rows



**Honourable mention to Leonardo DiCaprio who is my favourite :)**

In [28]:
spark.createDataFrame(martin_cast,'String')\
.toDF('actors').groupBy('actors').count().filter(col('actors')=='Leonardo DiCaprio').show()

+-----------------+-----+
|           actors|count|
+-----------------+-----+
|Leonardo DiCaprio|    2|
+-----------------+-----+



**2.3 Popular TV Show Directors**

In [29]:
# popular directors by tv shows

df_mod_tv_show.groupBy('director').count().\
orderBy(col('count').desc()).filter(col('director')!='null').show(5,0)

+-------------------+-----+
|director           |count|
+-------------------+-----+
|Alastair Fothergill|3    |
|Shin Won-ho        |2    |
|Rob Seidenglanz    |2    |
|Stan Lathan        |2    |
|Ken Burns          |2    |
+-------------------+-----+
only showing top 5 rows



**Looking at Alastair Fothergill**

In [30]:
# tv shows by Alastair Fothergill
df.filter(col('director')=='Alastair Fothergill').select('title','cast').show(3,0)

+------------------------------------------------+------------------+
|title                                           |cast              |
+------------------------------------------------+------------------+
|Frozen Planet                                   |David Attenborough|
|Planet Earth: The Complete Collection           |David Attenborough|
|The Blue Planet: A Natural History of the Oceans|David Attenborough|
+------------------------------------------------+------------------+



**2.4 Popular Movie Actors**

In [31]:
# extracting movie actors from cast

actor_movie = []

df_movie_actor_list = df_mod_movie.na.drop(subset='cast').withColumn('cast_mod',split(col('cast'),',')).select('cast_mod')

for num in range(0,df_movie_actor_list.count()):
    for i in df_movie_actor_list.collect()[num][0]:
        
        actor_movie.append(i)

In [32]:
df_movie_cast = spark.createDataFrame(actor_movie,StringType()).toDF('actor') 

df_movie_cast.groupBy('actor').count().orderBy(col('count').desc()).show()
    

+-----------------+-----+
|            actor|count|
+-----------------+-----+
|      Anupam Kher|   37|
|          Om Puri|   27|
|   Shah Rukh Khan|   27|
|      Boman Irani|   25|
|     Paresh Rawal|   24|
|     Akshay Kumar|   22|
|   Kareena Kapoor|   20|
|     Adam Sandler|   19|
|   Gulshan Grover|   18|
|   Yashpal Sharma|   18|
| Naseeruddin Shah|   18|
| Amitabh Bachchan|   18|
|           Asrani|   17|
|      John Cleese|   17|
|    Andrea Libman|   16|
|      Manoj Joshi|   16|
|       Ajay Devgn|   16|
|      Amrish Puri|   16|
|     Hassan Hosny|   16|
|       Vijay Raaz|   15|
+-----------------+-----+
only showing top 20 rows



**2.5 Popular TV Show Actors**

In [33]:
# extracting tv show actors from cast

df_tv_show_actor_list = df_mod_tv_show.na.drop(subset='cast').withColumn('cast_mod',split('cast',',')).select('cast_mod')

actor_tv_show = []

for num in range(0,df_tv_show_actor_list.count()):
    for i in df_tv_show_actor_list.collect()[num][0]:
        actor_tv_show.append(i)

In [34]:
df_tv_show_cast = spark.createDataFrame(actor_tv_show,StringType()).toDF('actor')

df_tv_show_cast.groupBy('actor').count().orderBy(col('count').desc()).show()

+------------------+-----+
|             actor|count|
+------------------+-----+
|  Takahiro Sakurai|   22|
|         Ai Kayano|   16|
|    Junichi Suwabe|   15|
|         Yuki Kaji|   15|
|       Daisuke Ono|   13|
|David Attenborough|   13|
|     Kana Hanazawa|   12|
|   Tomokazu Sugita|   12|
|  Yoshimasa Hosoya|   12|
|   Yuichi Nakamura|   11|
|      Vincent Tong|   11|
|    Hiroshi Kamiya|   10|
| Katsuyuki Konishi|   10|
|  Miyuki Sawashiro|   10|
|      Jun Fukuyama|    9|
|     Mamoru Miyano|    9|
|      Eri Kitamura|    9|
|     Kenjiro Tsuda|    9|
|  Nobuhiko Okamoto|    9|
|     Ashleigh Ball|    9|
+------------------+-----+
only showing top 20 rows



### 3. Looking at Genres

The modern-day consumer is willing to look past subtitles on the screen and discover content which is completely different to their native language making **international movies** a very popular genre

In [35]:
# extracting genres from listed_in

genre_list = df.withColumn('main_genre',split(col('listed_in'),',')).select('main_genre').na.drop()
genres = []
for num in range(0,genre_list.count()):
    for i in genre_list.collect()[num][0]:
        genres.append(i)


In [36]:
# popular genres

df_main_genre = spark.createDataFrame(genres,StringType()).toDF('genre')

df_main_genre.groupBy('genre').count().orderBy(col('count').desc()).show(5,0)



+---------------------+-----+
|genre                |count|
+---------------------+-----+
| International Movies|2322 |
|Dramas               |1380 |
|Comedies             |1070 |
|Documentaries        |750  |
|Action & Adventure   |720  |
+---------------------+-----+
only showing top 5 rows



aligining values correctly

In [37]:
df_main_genre.withColumn('genre',regexp_replace('genre',' Dramas','Dramas')).\
              withColumn('genre',regexp_replace('genre',' Comedies','Comedies')).\
              withColumn('genre',regexp_replace('genre',' Comedies','Comedies')).\
              withColumn('genre',regexp_replace('genre',' International Movies','International Movies')).\
              withColumn('genre',regexp_replace('genre',' Documentaries','Documentaries')).\
              withColumn('genre',regexp_replace('genre',' Action & Adventure','Action & Adventure')).\
              groupBy('genre').count().orderBy(col('count').desc()).show(5,0)

+--------------------+-----+
|genre               |count|
+--------------------+-----+
|International Movies|2434 |
|Dramas              |2100 |
|Comedies            |1467 |
|Documentaries       |784  |
|Action & Adventure  |720  |
+--------------------+-----+
only showing top 5 rows



### 4. Looking at Ratings

In [38]:
ratings = ('TV-14' ,'TV-PG', 'R' , 'PG-13' , 'TV-Y' , 'TV-Y7' , 'PG' , 'TV-G' , 'NR' , 'G' , 'TV-Y7-FV' , 'UR' , 'NC-17')

In [39]:
# popular ratings

df_mod_genre = df.filter((col('rating') =='TV-14') | (col('rating') =='TV-PG') | (col('rating') =='R') | \
                         (col('rating') =='PG-13') | (col('rating') == 'TV-Y')  | (col('rating') == 'TV-Y7')  | \
                         (col('rating') =='PG') | (col('rating') =='TV-G') | (col('rating') =='NR') | (col('rating') =='G') |\
                         (col('rating') =='TV-Y7-FV') | (col('rating') =='UR') | (col('rating') =='NC-17')).groupBy('rating').count()\
                 .orderBy(col('count').desc())

df_mod_genre.withColumn('Share%',round(col('count')/df_mod_genre.select(sum('count')).collect()[0][0]*100,2)).show(5)


+------+-----+------+
|rating|count|Share%|
+------+-----+------+
| TV-14| 1929| 39.28|
| TV-PG|  805| 16.39|
|     R|  663|  13.5|
| PG-13|  386|  7.86|
|  TV-Y|  280|   5.7|
+------+-----+------+
only showing top 5 rows



### 5. Top Countries by TV Show and Movies

**5.1 By TV Show**

In [40]:
# extracting countries from country list for tv show
df_mod_country_tv_show = df_mod_tv_show.withColumn('country_mod',split('country',',')).select('country_mod').na.drop()

country_list_tv_show = []

for num in range(0,df_mod_country_tv_show.count()):
    for i in df_mod_country_tv_show.collect()[num][0]:
        country_list_tv_show.append(i)

In [41]:
# countries with highest tv show content - top 3
df_country_tv_show = spark.createDataFrame(country_list_tv_show,StringType()).toDF('country')

total_country_tv_show_sum = df_country_tv_show.groupBy('country').count().agg({'count':'sum'}).collect()[0][0]

df_country_tv_show.groupBy('country').count().orderBy(col('count').desc()).\
withColumn('Share%',round(col('count')/total_country_tv_show_sum*100,2)).show(3,0)

+--------------+-----+------+
|country       |count|Share%|
+--------------+-----+------+
|United States |781  |31.83 |
|United Kingdom|236  |9.62  |
|Japan         |162  |6.6   |
+--------------+-----+------+
only showing top 3 rows



formatting country values correctly

In [42]:
df_country_tv_show.withColumn('country',regexp_replace('country',' United States','United States')).\
                   groupBy('country').count().orderBy(col('count').desc()).\
                   withColumn('Share%',round(col('count')/total_country_tv_show_sum*100,2)).show(3,0)

+--------------+-----+------+
|country       |count|Share%|
+--------------+-----+------+
|United States |864  |35.21 |
|United Kingdom|236  |9.62  |
|Japan         |162  |6.6   |
+--------------+-----+------+
only showing top 3 rows



**5.2 By Movies**

In [43]:
# extracting countries from country list for movies
df_mod_country_movie = df_mod_movie.withColumn('country_mod',split('country',',')).select('country_mod').na.drop()

country_list_movie = []

for num in range(0,df_mod_country_movie.count()):
    for i in df_mod_country_movie.collect()[num][0]:
        country_list_movie.append(i)

In [44]:
# countries with highest movie content - top 3
df_country_movie = spark.createDataFrame(country_list_movie,StringType()).toDF('country')

total_country_movie_sum = df_country_movie.groupBy('country').count().agg({'count':'sum'}).collect()[0][0]

df_country_movie.groupBy('country').count().orderBy(col('count').desc())\
                .withColumn('Share%',round(col('count')/total_country_movie_sum*100,2))\
                .withColumn('country',regexp_replace('country',' United States','United States')).show(3,0)

+--------------+-----+------+
|country       |count|Share%|
+--------------+-----+------+
|United States |2090 |31.61 |
|India         |883  |13.36 |
|United Kingdom|341  |5.16  |
+--------------+-----+------+
only showing top 3 rows



## 6. Looking at United Kingdom and India

In [45]:
#pandas data table for United Kingdom with type and year

df_mod_country_total = df_mod.withColumn('country_mod',split('country',','))\
                       .select('title','type','added_year','country_mod')\
                       .na.drop()


In [46]:
year_mod_list = []
type_mod_list = []
country_mod_list = []
title_mod_list = []
for num in range(1,df_mod_country_total.count()):
    for i in df_mod_country_total.collect()[num][3]:
        year_mod_list.append(df_mod_country_total.collect()[num][2])
        type_mod_list.append(df_mod_country_total.collect()[num][1])
        country_mod_list.append(i)
        title_mod_list.append(df_mod_country_total.collect()[num][0])

In [108]:
country_mod_spark.groupBy('country').pivot('type').count().na.drop().orderBy(col('Movie'))\
                 .filter((col('country')=='United Kingdom')|(col('country')=='India'))\
                 .withColumn('Total',col('Movie')+col('TV Show')).show()

+--------------+-----+-------+-----+
|       country|Movie|TV Show|Total|
+--------------+-----+-------+-----+
|United Kingdom|  341|    235|  576|
|         India|  883|     73|  956|
+--------------+-----+-------+-----+



### 6.1 United Kingdom
Looking at United Kingdom Yearly Growth

**6.1.1 Total Content Added Yearly**

In [47]:
import pandas as pd
data = [year_mod_list,type_mod_list,country_mod_list,title_mod_list]
country_year = pd.DataFrame(data).transpose().rename(columns=({2:'country',0:'year',1:'type',3:'title'}))

In [48]:
country_mod_spark = spark.createDataFrame(country_year)

In [49]:
country_mod_spark = country_mod_spark.select('year','type','country')

In [50]:
country_mod_spark.groupBy('country','year').count().filter(col('country')=='United Kingdom').orderBy('year').show()

+--------------+-----+-----+
|       country| year|count|
+--------------+-----+-----+
|United Kingdom| 2014|    3|
|United Kingdom| 2015|    4|
|United Kingdom| 2016|   45|
|United Kingdom| 2017|  119|
|United Kingdom| 2018|  117|
|United Kingdom| 2019|  160|
|United Kingdom| 2020|  122|
|United Kingdom| 2021|    6|
+--------------+-----+-----+



**6.1.2 TV Show Content Added Yearly**

In [51]:
# tv shows added over the years
country_mod_spark.filter(col('country')=='United Kingdom').filter(col('type')=='TV Show')\
                 .groupBy('country','year').count().orderBy('year').show()

+--------------+-----+-----+
|       country| year|count|
+--------------+-----+-----+
|United Kingdom| 2015|    3|
|United Kingdom| 2016|   20|
|United Kingdom| 2017|   49|
|United Kingdom| 2018|   42|
|United Kingdom| 2019|   61|
|United Kingdom| 2020|   58|
|United Kingdom| 2021|    2|
+--------------+-----+-----+



**6.1.3 Movie Content Added Yearly**

In [52]:
# movies added over the years
country_mod_spark.filter(col('country')=='United Kingdom').filter(col('type')=='Movie')\
                 .groupBy('country','year').count().orderBy('year').show()

+--------------+-----+-----+
|       country| year|count|
+--------------+-----+-----+
|United Kingdom| 2014|    3|
|United Kingdom| 2015|    1|
|United Kingdom| 2016|   25|
|United Kingdom| 2017|   70|
|United Kingdom| 2018|   75|
|United Kingdom| 2019|   99|
|United Kingdom| 2020|   64|
|United Kingdom| 2021|    4|
+--------------+-----+-----+



### 6.2 India

**6.2.1 Total Content Added Yearly**

In [53]:
#content added over the years
country_mod_spark.groupBy('country','year').count().filter(col('country')=='India').orderBy('year').show()

+-------+-----+-----+
|country| year|count|
+-------+-----+-----+
|  India| 2016|   12|
|  India| 2017|  154|
|  India| 2018|  346|
|  India| 2019|  235|
|  India| 2020|  196|
|  India| 2021|   13|
+-------+-----+-----+



**6.2.2 TV Show Added Yearly**

In [54]:
country_mod_spark.filter(col('country')=='India').filter(col('type')=='TV Show')\
                 .groupBy('country','year').count().orderBy('year').show()

+-------+-----+-----+
|country| year|count|
+-------+-----+-----+
|  India| 2017|   14|
|  India| 2018|   20|
|  India| 2019|   19|
|  India| 2020|   19|
|  India| 2021|    1|
+-------+-----+-----+



**6.2.3 Movie Added Yearly**

In [55]:
# movies added over the years
country_mod_spark.filter(col('country')=='India').filter(col('type')=='Movie')\
                 .groupBy('country','year').count().orderBy('year').show()

+-------+-----+-----+
|country| year|count|
+-------+-----+-----+
|  India| 2016|   12|
|  India| 2017|  140|
|  India| 2018|  326|
|  India| 2019|  216|
|  India| 2020|  177|
|  India| 2021|   12|
+-------+-----+-----+



## 7. Predicting Yearly Content Added

We will take the years from 2016 to 2020

In [56]:
df_ml = df_mod.select('added_year').groupBy('added_year').count().orderBy(col('added_year'))\
              .na.drop().filter((col('added_year') > 2015) & (col('added_year') < 2021))

In [57]:
df_ml.show()

+----------+-----+
|added_year|count|
+----------+-----+
|      2016|  443|
|      2017| 1223|
|      2018| 1683|
|      2019| 2151|
|      2020| 2003|
+----------+-----+



importing libraries needed

In [58]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [77]:
df_ml = df_ml.withColumn('added_year',col('added_year').cast('Integer'))

In [78]:
assembler = VectorAssembler(inputCols=['added_year'],outputCol='features')

In [79]:
output = assembler.transform(df_ml)
final_data = output.select('features','count')

defining our train and test data set
<br>
defining our linear model

In [80]:
train_data,test_data = final_data.randomSplit([0.7,0.3])
lr = LinearRegression(labelCol='count')
lr_model = lr.fit(train_data)
test_results = lr_model.evaluate(test_data)

In [81]:
unlabeled_data = test_data.select('features')
predictions = lr_model.transform(unlabeled_data)

predicting values based on features

In [82]:
predictions.show()

+--------+------------------+
|features|        prediction|
+--------+------------------+
|[2020.0]|2771.0000002649613|
+--------+------------------+



In [83]:
df_ml.show()

+----------+-----+
|added_year|count|
+----------+-----+
|      2016|  443|
|      2017| 1223|
|      2018| 1683|
|      2019| 2151|
|      2020| 2003|
+----------+-----+



% growth from 2019 to 2020 based on predicted value

In [1]:
(2771/2151-1)*100

28.82380288238029

projected growth for 2021

In [2]:
(2003*128)/100

2563.84

resulting coefificient

In [86]:
lr_model.coefficients

DenseVector([558.4])